In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# *Text Reduction*
___
#### This project aims to reduce texts from the internet using NLP techniques.

# Packages :

In [5]:
!pip install beautifulsoup4

In [6]:
import nltk
from nltk.tokenize import  word_tokenize,sent_tokenize
import re
import requests
from bs4 import BeautifulSoup
import string

stopwords = nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords.append("href=")
stopwords.append("class=")
stopwords.append("id=")
stopwords.append("title=")

In [8]:
ponc = string.punctuation + "  '' ,\"\""

In [9]:
ponc

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~  \'\' ,""'

## Get articals:

In [10]:
url = "https://en.wikipedia.org/wiki/Artificial_intelligence"

In [11]:
def weghts_extracting(url):
    
    responce = requests.get(url)
    
    html = responce.text
    html_obj = BeautifulSoup(html,'html.parser')
    
    # find all paragraphs
    all_paragraph = html_obj.find_all("p")
    
    # remove html tags
    pattren = "\<\/?.{1,4}\>"
    pattren2 = "\<a.{0,100}\>"
    
    all_paragraph = [re.sub(pattren,"",str(i)) for i in all_paragraph]
    all_paragraph = [re.sub(pattren2,"",str(i)) for i in all_paragraph]
    clean_txt = all_paragraph
    
    # tokenize words
    all_paragraph = [word_tokenize(txt.lower()) for txt in all_paragraph]
    
    # remove punctuation
    all_paragraph = [[word for word in txt if word not in ponc] for txt in all_paragraph]
    
    # remove stopwords
    all_paragraph = [[word for word in txt if word not in stopwords] for txt in all_paragraph]
    
    # words freq
    word_fr = {}

    for txt in all_paragraph:
        for word in txt:
            if word in word_fr.keys():
                word_fr[word] += 1
            else:
                word_fr[word] = 1
                
    # get max freq
    max_val = max(word_fr.values())
    
    # normalizeation
    word_fr_nrml = {}
    for key,val in word_fr.items():
        word_fr_nrml[key] = val/max_val
        
    return clean_txt , word_fr_nrml


In [12]:
clean_text, word_fr = weghts_extracting(url)

In [41]:
def text_reduction(clean_text,word_frq,threshold=2):
    
    tokenized = [sent_tokenize(txt) for txt in clean_text]
    
    
    sents_res = {}
    for txt in tokenized:
        for sent in txt:
            sents_res[sent] = 0
            for word in word_tokenize(sent):
                if word in word_frq.keys():
                    sents_res[sent] += word_frq[word]
                    
    sents_res_keys = []
    sents_res_values = np.array([], dtype="float64")

    for k,v in sents_res.items():
        sents_res_keys.append(k)
        sents_res_values = np.append(sents_res_values, v)
        
    threshold_prc = threshold*max(sents_res_values)/100
    
    for cnt in range(len(sents_res)):
        if sents_res_values[cnt] >=threshold_prc:
            print(sents_res_keys[cnt])


In [42]:
text_reduction(clean_text, word_fr,80)

Alan Turing was the first person to conduct substantial research in the field that he called machine intelligence.<sup class="reference" id="cite_ref-turing_5-0">[5] Artificial intelligence was founded as an academic discipline in 1956,<sup class="reference" id="cite_ref-Dartmouth_workshop_6-0">[6] by those now considered the founding fathers of AI: John McCarthy, Marvin Minksy, <a href="/wiki/Nathaniel_Rochester_(computer_scientist)" title="Nathaniel Rochester (computer scientist)">Nathaniel Rochester, and Claude Shannon.<sup class="reference" id="cite_ref-7">[8] The field went through multiple cycles of optimism,<sup class="reference" id="cite_ref-AI_in_the_60s_9-0">[10] followed by periods of disappointment and loss of funding, known as AI winter.<sup class="reference" id="cite_ref-First_AI_winter_11-0">[12] Funding and interest vastly increased after 2012 when deep learning surpassed all previous AI techniques,<sup class="reference" id="cite_ref-Deep_learning_revolution_13-0">[13] 